# EL TIEMPO:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [1]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00


**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [2]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/El tiempo'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**BIGRAMAS:**

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [4]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [5]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

/content/drive/MyDrive/Medios/El tiempo/eltiempo1.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo10.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo11.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo12.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo13.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo14.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo15.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo2.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo3.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo4.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo5.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo6.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo7.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo8.txt
/content/drive/MyDrive/Medios/El tiempo/eltiempo9.txt
/content/drive/MyDrive/Medios/El tiempo/Eltiempo24.txt
/content/drive/MyDrive/Medios/El tiempo/Eltiempo23.txt
/content/drive/MyDrive/Medios/El tiempo/Eltiempo22.txt
/content/drive/MyDr

## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  # print(f'PARA {filename}')
  # print('Centralidad de grado:')
  # print(degree_centrality)
  # print('Centralidad de intermediación')
  # print(betweenness_centrality)
  # print('Centralidad de cercanía')
  # print(closeness_centrality)
  # print('Centralidad de vector propio')
  # print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

PARA Publimetro53.txt
Centralidad de grado:
{'cogieron': 0.005780346820809248, 'presunto': 0.017341040462427744, 'feminicida': 0.03468208092485549, 'apunalo': 0.011560693641618497, 'menor': 0.03468208092485549, 'anos': 0.04046242774566474, 'meta': 0.04046242774566474, 'gobernadora': 0.028901734104046242, 'departamento': 0.023121387283236993, 'rafaela': 0.017341040462427744, 'cortes': 0.017341040462427744, 'confirmo': 0.023121387283236993, 'captura': 0.028901734104046242, 'senalado': 0.023121387283236993, 'asesino': 0.011560693641618497, 'quedo': 0.011560693641618497, 'paralizado': 0.011560693641618497, 'despues': 0.03468208092485549, 'conocio': 0.011560693641618497, 'asesinato': 0.011560693641618497, 'sharit': 0.017341040462427744, 'aponte': 0.017341040462427744, 'municipio': 0.011560693641618497, 'granada': 0.046242774566473986, 'hecho': 0.03468208092485549, 'genero': 0.011560693641618497, 'indignacion': 0.023121387283236993, 'nacional': 0.011560693641618497, 'llevo': 0.01156069364161

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Tiempo/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
Eltiempo16.txt                130.0  0.020274  0.009956  0.007752  0.015504   
Eltiempo17.txt                234.0  0.011225  0.008498  0.004292  0.008584   
Eltiempo18.txt                151.0  0.016336  0.007763  0.006667  0.013333   
Eltiempo19.txt                224.0  0.011211  0.006755  0.004484  0.008969   
Eltiempo20.txt                170.0  0.016220  0.009619  0.005917  0.011834   
Eltiempo21.txt                215.0  0.012041  0.007118  0.004673  0.009346   
Eltiempo22.txt                221.0  0.012505  0.007602  0.004545  0.009091   
Eltiempo23.txt                134.0  0.019526  0.009317  0.007519  0.015038   
Eltiempo24.txt                 67.0  0.036183  0.013945  0.015152  0.030303   
eltiempo1.txt                 116.0  0.022039  0.011

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Eltiempo16.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.416827   
Centralidad de Cercania                    0.432189   
Centralidad de Vector Propio               0.718498   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.416827   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.248445   
Centralidad de Vector Propio                        0.184581   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.432189   
Centralidad de Intermediacion                 0.248445   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.612349   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'eltiempo1.txt': {'Tamaño': 116, 'Densidad': 0.022038980509745128}, 'eltiempo10.txt': {'Tamaño': 140, 'Densidad': 0.01778006166495375}, 'eltiempo11.txt': {'Tamaño': 276, 'Densidad': 0.00919631093544137}, 'eltiempo12.txt': {'Tamaño': 516, 'Densidad': 0.005599458116956424}, 'eltiempo13.txt': {'Tamaño': 551, 'Densidad': 0.005055271407358522}, 'eltiempo14.txt': {'Tamaño': 520, 'Densidad': 0.005180080035571365}, 'eltiempo15.txt': {'Tamaño': 154, 'Densidad': 0.01621254562431033}, 'eltiempo2.txt': {'Tamaño': 117, 'Densidad': 0.02180960801650457}, 'eltiempo3.txt': {'Tamaño': 276, 'Densidad': 0.010355731225296443}, 'eltiempo4.txt': {'Tamaño': 212, 'Densidad': 0.012697844943217384}, 'eltiempo5.txt': {'Tamaño': 224, 'Densidad': 0.011010570147341447}, 'eltiempo6.txt': {'Tamaño': 154, 'Densidad': 0.014854426619132501}, 'eltiempo7.txt': {'Tamaño': 150, 'Densidad': 0.017002237136465325}, 'eltiempo8.txt': {'Tamaño': 171, 'Densidad': 0.01582387340901273}, 'eltiempo9.txt': {'Tamaño': 243, 'Densidad': 0

### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

         Noticia  Palabra  Centralidad de Grado  \
0  eltiempo1.txt   bogota              0.226087   
1  eltiempo1.txt   envian              0.034783   
2  eltiempo1.txt   carcel              0.078261   
3  eltiempo1.txt   hombre              0.243478   
4  eltiempo1.txt  acusado              0.052174   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.184585                 0.463710   
1                       0.000554                 0.368590   
2                       0.019923                 0.410714   
3                       0.250023                 0.477178   
4                       0.001721                 0.368590   

   Centralidad de Vector Propio  
0                      0.300635  
1                      0.078516  
2                      0.132639  
3                      0.265212  
4                      0.098199  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Tiempo/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
Eltiempo16.txt                130.0  0.060346  0.027690  0.023256  0.046512   
Eltiempo17.txt                234.0  0.033418  0.024051  0.012876  0.025751   
Eltiempo18.txt                151.0  0.049536  0.023845  0.020000  0.040000   
Eltiempo19.txt                224.0  0.034153  0.021527  0.013453  0.026906   
Eltiempo20.txt                170.0  0.048590  0.028346  0.017751  0.035503   
Eltiempo21.txt                215.0  0.036601  0.021983  0.014019  0.028037   
Eltiempo22.txt                221.0  0.037351  0.021910  0.013636  0.027273   
Eltiempo23.txt                134.0  0.059140  0.029788  0.022556  0.045113   
Eltiempo24.txt                 67.0  0.110357  0.043644  0.045455  0.090909   
eltiempo1.txt                 116.0  0.066417  0.034

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Eltiempo16.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.577203   
Centralidad de Cercania                    0.542750   
Centralidad de Vector Propio               0.803592   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.577203   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.425142   
Centralidad de Vector Propio                        0.217614   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.542750   
Centralidad de Intermediacion                 0.425142   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.668132   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_s)

{'eltiempo1.txt': {'Tamaño': 116, 'Densidad': 0.0664167916041979}, 'eltiempo10.txt': {'Tamaño': 140, 'Densidad': 0.05313463514902364}, 'eltiempo11.txt': {'Tamaño': 276, 'Densidad': 0.027457180500658762}, 'eltiempo12.txt': {'Tamaño': 516, 'Densidad': 0.016700534356890194}, 'eltiempo13.txt': {'Tamaño': 551, 'Densidad': 0.015139415937964033}, 'eltiempo14.txt': {'Tamaño': 520, 'Densidad': 0.015495775900400178}, 'eltiempo15.txt': {'Tamaño': 154, 'Densidad': 0.048467871997283765}, 'eltiempo2.txt': {'Tamaño': 117, 'Densidad': 0.06542882404951371}, 'eltiempo3.txt': {'Tamaño': 276, 'Densidad': 0.030408432147562583}, 'eltiempo4.txt': {'Tamaño': 212, 'Densidad': 0.038987749262273094}, 'eltiempo5.txt': {'Tamaño': 224, 'Densidad': 0.03343209481101858}, 'eltiempo6.txt': {'Tamaño': 154, 'Densidad': 0.04507257448433919}, 'eltiempo7.txt': {'Tamaño': 150, 'Densidad': 0.051543624161073824}, 'eltiempo8.txt': {'Tamaño': 171, 'Densidad': 0.046714826281389746}, 'eltiempo9.txt': {'Tamaño': 243, 'Densidad': 0.

# EL COLOMBIANO:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/El colombiano'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/El colombiano/ElColombiano1.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano2.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano3.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano4.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano5.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano6.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano7.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano8.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano9.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano10.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano11.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano12.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano13.txt
/content/drive/MyDrive/Medios/El colombiano/ElColombiano14.txt


## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Colombiano/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

                   Centralidad de Grado                                \
                                  count      mean       std       min   
Noticia                                                                 
ElColombiano1.txt                  79.0  0.030185  0.013623  0.012821   
ElColombiano10.txt                208.0  0.012588  0.007008  0.004831   
ElColombiano11.txt                289.0  0.009251  0.006940  0.003472   
ElColombiano12.txt                161.0  0.016460  0.007896  0.012500   
ElColombiano13.txt                128.0  0.020177  0.011681  0.007874   
ElColombiano14.txt                170.0  0.014271  0.006614  0.005917   
ElColombiano2.txt                 131.0  0.020787  0.012057  0.007692   
ElColombiano3.txt                 219.0  0.012316  0.006801  0.004587   
ElColombiano4.txt                 219.0  0.012316  0.006801  0.004587   
ElColombiano5.txt                 180.0  0.013222  0.005961  0.005587   
ElColombiano6.txt                 127.0  0.018373  

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para ElColombiano1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.657519   
Centralidad de Cercania                    0.468169   
Centralidad de Vector Propio               0.735949   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.657519   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.259561   
Centralidad de Vector Propio                        0.421387   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.468169   
Centralidad de Intermediacion                 0.259561   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.652288   

                               Centralidad de Vector Propio  
Cent

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'ElColombiano1.txt': {'Tamaño': 79, 'Densidad': 0.03018500486854917}, 'ElColombiano2.txt': {'Tamaño': 131, 'Densidad': 0.020786846741045216}, 'ElColombiano3.txt': {'Tamaño': 219, 'Densidad': 0.01231619957270328}, 'ElColombiano4.txt': {'Tamaño': 219, 'Densidad': 0.01231619957270328}, 'ElColombiano5.txt': {'Tamaño': 180, 'Densidad': 0.013221601489757914}, 'ElColombiano6.txt': {'Tamaño': 127, 'Densidad': 0.01837270341207349}, 'ElColombiano7.txt': {'Tamaño': 356, 'Densidad': 0.0073112834309226144}, 'ElColombiano8.txt': {'Tamaño': 116, 'Densidad': 0.02638680659670165}, 'ElColombiano9.txt': {'Tamaño': 116, 'Densidad': 0.02638680659670165}, 'ElColombiano10.txt': {'Tamaño': 208, 'Densidad': 0.012588257153474544}, 'ElColombiano11.txt': {'Tamaño': 289, 'Densidad': 0.009251249519415609}, 'ElColombiano12.txt': {'Tamaño': 161, 'Densidad': 0.016459627329192546}, 'ElColombiano13.txt': {'Tamaño': 128, 'Densidad': 0.020177165354330708}, 'ElColombiano14.txt': {'Tamaño': 170, 'Densidad': 0.0142707970762

### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

             Noticia      Palabra  Centralidad de Grado  \
0  ElColombiano1.txt         tras              0.038462   
1  ElColombiano1.txt  feminicidio              0.256410   
2  ElColombiano1.txt        diana              0.102564   
3  ElColombiano1.txt     carolina              0.115385   
4  ElColombiano1.txt        serna              0.128205   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.000000                 0.296578   
1                       0.196314                 0.410526   
2                       0.011168                 0.357798   
3                       0.014102                 0.359447   
4                       0.050085                 0.390000   

   Centralidad de Vector Propio  
0                      0.073085  
1                      0.287356  
2                      0.179400  
3                      0.190173  
4                      0.214766  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Colombiano/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

                   Centralidad de Grado                                \
                                  count      mean       std       min   
Noticia                                                                 
ElColombiano1.txt                  79.0  0.091853  0.040356  0.038462   
ElColombiano10.txt                208.0  0.036929  0.019835  0.014493   
ElColombiano11.txt                289.0  0.027345  0.019479  0.010417   
ElColombiano12.txt                161.0  0.050621  0.025313  0.025000   
ElColombiano13.txt                128.0  0.061270  0.035199  0.023622   
ElColombiano14.txt                170.0  0.042882  0.018908  0.017751   
ElColombiano2.txt                 131.0  0.062478  0.036257  0.023077   
ElColombiano3.txt                 219.0  0.037535  0.021495  0.013761   
ElColombiano4.txt                 219.0  0.037535  0.021495  0.013761   
ElColombiano5.txt                 180.0  0.039913  0.019281  0.016760   
ElColombiano6.txt                 127.0  0.055493  

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para ElColombiano1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.783990   
Centralidad de Cercania                    0.595176   
Centralidad de Vector Propio               0.804624   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.783990   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.396046   
Centralidad de Vector Propio                        0.463110   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.595176   
Centralidad de Intermediacion                 0.396046   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.813195   

                               Centralidad de Vector Propio  
Cent

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

# EL ESPECTADOR:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/El espectador'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/El espectador/Elespectador3.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador4.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador5.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador5 (1).txt
/content/drive/MyDrive/Medios/El espectador/Elespectador6.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador7.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador8.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador9.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador10.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador11.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador12.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador13.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador14.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador15.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador16.txt
/content/drive/MyDrive/Medios/El espectador/Elespectador17.

## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Espectador/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

                      Centralidad de Grado                                \
                                     count      mean       std       min   
Noticia                                                                    
Elespectador10.txt                   204.0  0.012170  0.005711  0.009852   
Elespectador11.txt                   206.0  0.012692  0.007598  0.009756   
Elespectador12.txt                   507.0  0.005800  0.004561  0.001976   
Elespectador13.txt                   145.0  0.017912  0.010244  0.013889   
Elespectador14.txt                   126.0  0.018286  0.007426  0.016000   
Elespectador15.txt                   400.0  0.006717  0.004906  0.002506   
Elespectador16.txt                   274.0  0.009171  0.004747  0.007326   
Elespectador17.txt                   394.0  0.006665  0.004270  0.005089   
Elespectador18.txt                   203.0  0.012291  0.006080  0.004950   
Elespectador3.txt                    383.0  0.007163  0.005164  0.005236   
Elespectador

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Elespectador10.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.803682   
Centralidad de Cercania                    0.473768   
Centralidad de Vector Propio               0.584234   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.803682   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.606047   
Centralidad de Vector Propio                        0.722279   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.473768   
Centralidad de Intermediacion                 0.606047   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.747852   

                               Centralidad de Vector Propio  
Cen

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'Elespectador3.txt': {'Tamaño': 383, 'Densidad': 0.0071630691837655325}, 'Elespectador4.txt': {'Tamaño': 601, 'Densidad': 0.0045535219079312255}, 'Elespectador5.txt': {'Tamaño': 315, 'Densidad': 0.008998079061773329}, 'Elespectador5 (1).txt': {'Tamaño': 315, 'Densidad': 0.008998079061773329}, 'Elespectador6.txt': {'Tamaño': 237, 'Densidad': 0.009440034327397553}, 'Elespectador7.txt': {'Tamaño': 494, 'Densidad': 0.006093404833663187}, 'Elespectador8.txt': {'Tamaño': 185, 'Densidad': 0.013807285546415981}, 'Elespectador9.txt': {'Tamaño': 325, 'Densidad': 0.008395061728395062}, 'Elespectador10.txt': {'Tamaño': 204, 'Densidad': 0.012170385395537525}, 'Elespectador11.txt': {'Tamaño': 206, 'Densidad': 0.01269239876864788}, 'Elespectador12.txt': {'Tamaño': 507, 'Densidad': 0.005800219847042589}, 'Elespectador13.txt': {'Tamaño': 145, 'Densidad': 0.017911877394636014}, 'Elespectador14.txt': {'Tamaño': 126, 'Densidad': 0.018285714285714287}, 'Elespectador15.txt': {'Tamaño': 400, 'Densidad': 0.0

### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

             Noticia       Palabra  Centralidad de Grado  \
0  Elespectador3.txt      stefanny              0.075916   
1  Elespectador3.txt      barranco              0.083770   
2  Elespectador3.txt         cinco              0.013089   
3  Elespectador3.txt  feminicidios              0.123037   
4  Elespectador3.txt       centros              0.065445   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.066283                 0.432127   
1                       0.052417                 0.434585   
2                       0.000000                 0.350781   
3                       0.119847                 0.431638   
4                       0.029983                 0.374143   

   Centralidad de Vector Propio  
0                      0.211502  
1                      0.249401  
2                      0.066554  
3                      0.225928  
4                      0.110536  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/El Espectador/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

                      Centralidad de Grado                                \
                                     count      mean       std       min   
Noticia                                                                    
Elespectador10.txt                   204.0  0.036318  0.017070  0.024631   
Elespectador11.txt                   206.0  0.037698  0.021070  0.019512   
Elespectador12.txt                   507.0  0.017424  0.013554  0.007905   
Elespectador13.txt                   145.0  0.052969  0.029200  0.027778   
Elespectador14.txt                   126.0  0.054857  0.022846  0.032000   
Elespectador15.txt                   400.0  0.020301  0.014621  0.007519   
Elespectador16.txt                   274.0  0.027593  0.013948  0.014652   
Elespectador17.txt                   394.0  0.020201  0.013121  0.012723   
Elespectador18.txt                   203.0  0.036921  0.018181  0.014851   
Elespectador3.txt                    383.0  0.022145  0.017198  0.013089   
Elespectador

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Elespectador10.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.920355   
Centralidad de Cercania                    0.602178   
Centralidad de Vector Propio               0.701596   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.920355   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.592772   
Centralidad de Vector Propio                        0.606107   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.602178   
Centralidad de Intermediacion                 0.592772   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.824065   

                               Centralidad de Vector Propio  
Cen

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

# VOLCÁNICAS:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/Volcanicas'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/Volcanicas/vocanicas1.txt
/content/drive/MyDrive/Medios/Volcanicas/vocanicas2.txt
/content/drive/MyDrive/Medios/Volcanicas/vocanicas3.txt
/content/drive/MyDrive/Medios/Volcanicas/vocanicas4.txt


## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/Volcánicas/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
vocanicas1.txt                472.0  0.006513  0.006158  0.002123  0.004246   
vocanicas2.txt                141.0  0.017832  0.011314  0.007143  0.014286   
vocanicas3.txt                225.0  0.011865  0.008440  0.004464  0.008929   
vocanicas4.txt                314.0  0.008852  0.008400  0.006390  0.006390   

                                             Centralidad de Intermediacion  \
                     50%       75%       max                         count   
Noticia                                                                      
vocanicas1.txt  0.004246  0.006369  0.050955                         472.0   
vocanicas2.txt  0.014286  0.014286  0.121429                         141.0   
vocanicas3.txt  0.008929  0.008929  0.093750            

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para vocanicas1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.924279   
Centralidad de Cercania                    0.559441   
Centralidad de Vector Propio               0.820022   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.924279   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.507389   
Centralidad de Vector Propio                        0.819740   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.559441   
Centralidad de Intermediacion                 0.507389   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.736584   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'vocanicas1.txt': {'Tamaño': 472, 'Densidad': 0.0065133685990859695}, 'vocanicas2.txt': {'Tamaño': 141, 'Densidad': 0.017831813576494427}, 'vocanicas3.txt': {'Tamaño': 225, 'Densidad': 0.011865079365079365}, 'vocanicas4.txt': {'Tamaño': 314, 'Densidad': 0.008852078712276917}}


### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

          Noticia       Palabra  Centralidad de Grado  \
0  vocanicas1.txt          tres              0.006369   
1  vocanicas1.txt  feminicidios              0.127389   
2  vocanicas1.txt        bogota              0.093418   
3  vocanicas1.txt         menos              0.025478   
4  vocanicas1.txt         horas              0.010616   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.000000                 0.334755   
1                       0.101194                 0.442669   
2                       0.067787                 0.452885   
3                       0.008184                 0.369122   
4                       0.000040                 0.354669   

   Centralidad de Vector Propio  
0                      0.021688  
1                      0.186008  
2                      0.173276  
3                      0.045542  
4                      0.034095  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/Volcánicas/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
vocanicas1.txt                472.0  0.019666  0.018903  0.006369  0.012739   
vocanicas2.txt                141.0  0.052178  0.030639  0.021429  0.042857   
vocanicas3.txt                225.0  0.034563  0.023149  0.017857  0.026786   
vocanicas4.txt                314.0  0.026414  0.023147  0.012780  0.019169   

                                             Centralidad de Intermediacion  \
                     50%       75%       max                         count   
Noticia                                                                      
vocanicas1.txt  0.012739  0.019108  0.188960                         472.0   
vocanicas2.txt  0.042857  0.042857  0.328571                         141.0   
vocanicas3.txt  0.026786  0.026786  0.241071            

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para vocanicas1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.943882   
Centralidad de Cercania                    0.678650   
Centralidad de Vector Propio               0.911605   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.943882   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.584089   
Centralidad de Vector Propio                        0.850368   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.678650   
Centralidad de Intermediacion                 0.584089   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.853676   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

# LA SILLA VACÍA:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/La silla vacía'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/La silla vacía/Silla1.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla2.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla3.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla4.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla5.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla6.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla7.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla11.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla12.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla13.txt
/content/drive/MyDrive/Medios/La silla vacía/Silla14.txt


## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/La Silla Vacía/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

            Centralidad de Grado                                          \
                           count      mean       std       min       25%   
Noticia                                                                    
Silla1.txt                 392.0  0.007581  0.006943  0.002558  0.005115   
Silla11.txt                 76.0  0.032982  0.016217  0.013333  0.026667   
Silla12.txt                 74.0  0.036653  0.020420  0.027397  0.027397   
Silla13.txt                565.0  0.004751  0.003414  0.001773  0.003546   
Silla14.txt                117.0  0.019304  0.007217  0.008621  0.017241   
Silla2.txt                 276.0  0.008775  0.006538  0.003636  0.007273   
Silla3.txt                 336.0  0.007516  0.006141  0.002985  0.005970   
Silla4.txt                 279.0  0.009489  0.007065  0.003597  0.007194   
Silla5.txt                  61.0  0.040437  0.014421  0.033333  0.033333   
Silla6.txt                 230.0  0.011088  0.005794  0.004367  0.008734   
Silla7.txt  

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Silla1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.934119   
Centralidad de Cercania                    0.496525   
Centralidad de Vector Propio               0.693546   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.934119   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.492822   
Centralidad de Vector Propio                        0.714384   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.496525   
Centralidad de Intermediacion                 0.492822   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.632886   

                               Centralidad de Vector Propio  
Centralidad

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'Silla1.txt': {'Tamaño': 392, 'Densidad': 0.007581293386919985}, 'Silla2.txt': {'Tamaño': 276, 'Densidad': 0.008774703557312254}, 'Silla3.txt': {'Tamaño': 336, 'Densidad': 0.007515991471215352}, 'Silla4.txt': {'Tamaño': 279, 'Densidad': 0.00948918284727057}, 'Silla5.txt': {'Tamaño': 61, 'Densidad': 0.040437158469945354}, 'Silla6.txt': {'Tamaño': 230, 'Densidad': 0.011087905828745016}, 'Silla7.txt': {'Tamaño': 383, 'Densidad': 0.007573168564515467}, 'Silla11.txt': {'Tamaño': 76, 'Densidad': 0.03298245614035088}, 'Silla12.txt': {'Tamaño': 74, 'Densidad': 0.036653091447611995}, 'Silla13.txt': {'Tamaño': 565, 'Densidad': 0.004751145421452331}, 'Silla14.txt': {'Tamaño': 117, 'Densidad': 0.019304450338933098}}


### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

      Noticia Palabra  Centralidad de Grado  Centralidad de Intermediacion  \
0  Silla1.txt    vida              0.061381                       0.022707   
1  Silla1.txt  muerte              0.025575                       0.012166   
2  Silla1.txt  lineas              0.191816                       0.180831   
3  Silla1.txt   rojas              0.148338                       0.090851   
4  Silla1.txt   zonas              0.056266                       0.012594   

   Centralidad de Cercania  Centralidad de Vector Propio  
0                 0.402678                      0.125803  
1                 0.359375                      0.059386  
2                 0.485714                      0.343449  
3                 0.460542                      0.300883  
4                 0.396552                      0.130867  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/La Silla Vacía/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

            Centralidad de Grado                                          \
                           count      mean       std       min       25%   
Noticia                                                                    
Silla1.txt                 392.0  0.022274  0.020289  0.010230  0.015345   
Silla11.txt                 76.0  0.097193  0.047743  0.040000  0.080000   
Silla12.txt                 74.0  0.107738  0.057555  0.054795  0.082192   
Silla13.txt                565.0  0.014266  0.010533  0.005319  0.010638   
Silla14.txt                117.0  0.058061  0.023237  0.025862  0.051724   
Silla2.txt                 276.0  0.026271  0.020399  0.010909  0.021818   
Silla3.txt                 336.0  0.022406  0.018039  0.014925  0.017910   
Silla4.txt                 279.0  0.028493  0.022905  0.010791  0.021583   
Silla5.txt                  61.0  0.119126  0.043123  0.066667  0.100000   
Silla6.txt                 230.0  0.033150  0.017481  0.013100  0.026201   
Silla7.txt  

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Silla1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.947298   
Centralidad de Cercania                    0.651896   
Centralidad de Vector Propio               0.873389   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.947298   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.573759   
Centralidad de Vector Propio                        0.793336   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.651896   
Centralidad de Intermediacion                 0.573759   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.845989   

                               Centralidad de Vector Propio  
Centralidad

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

# LA CASA DE LA MUJER:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/Casa de la mujer'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/Casa de la mujer/casa1.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa43.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa44.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa45.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa46.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa47.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa48.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa49.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa5.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa50.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa51.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa52.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa53.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa54.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa55.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa56.txt
/content/drive/MyDrive/Medios/Casa de la mujer/casa57.txt
/content/drive/M

## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/La Casa de la Mujer/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

           Centralidad de Grado                                          \
                          count      mean       std       min       25%   
Noticia                                                                   
casa1.txt                 441.0  0.006287  0.005069  0.002273  0.004545   
casa10.txt                251.0  0.009880  0.004176  0.004000  0.008000   
casa11.txt                149.0  0.017595  0.015146  0.013514  0.013514   
casa12.txt                 98.0  0.025668  0.017279  0.010309  0.020619   
casa13.txt                201.0  0.012687  0.006800  0.005000  0.010000   
...                         ...       ...       ...       ...       ...   
casa7.txt                 145.0  0.017337  0.007506  0.013889  0.013889   
casa70.txt                192.0  0.013416  0.007180  0.005236  0.010471   
casa71.txt                239.0  0.010935  0.005456  0.008403  0.008403   
casa8.txt                 256.0  0.009620  0.006249  0.003922  0.007843   
casa9.txt                

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para casa1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.910922   
Centralidad de Cercania                    0.466904   
Centralidad de Vector Propio               0.687963   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.910922   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.466671   
Centralidad de Vector Propio                        0.597509   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.466904   
Centralidad de Intermediacion                 0.466671   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.604900   

                               Centralidad de Vector Propio  
Centralidad 

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'casa1.txt': {'Tamaño': 441, 'Densidad': 0.006287363430220573}, 'casa43.txt': {'Tamaño': 250, 'Densidad': 0.010313253012048192}, 'casa44.txt': {'Tamaño': 664, 'Densidad': 0.004633920296570899}, 'casa45.txt': {'Tamaño': 118, 'Densidad': 0.02071563088512241}, 'casa46.txt': {'Tamaño': 119, 'Densidad': 0.02335849594074918}, 'casa47.txt': {'Tamaño': 188, 'Densidad': 0.0132552053703493}, 'casa48.txt': {'Tamaño': 287, 'Densidad': 0.008698618454716017}, 'casa49.txt': {'Tamaño': 116, 'Densidad': 0.020389805097451273}, 'casa5.txt': {'Tamaño': 127, 'Densidad': 0.018997625296837895}, 'casa50.txt': {'Tamaño': 232, 'Densidad': 0.01238990894163308}, 'casa51.txt': {'Tamaño': 77, 'Densidad': 0.031784005468216}, 'casa52.txt': {'Tamaño': 402, 'Densidad': 0.007059465763452066}, 'casa53.txt': {'Tamaño': 444, 'Densidad': 0.005795863583673968}, 'casa54.txt': {'Tamaño': 219, 'Densidad': 0.01265133425495371}, 'casa55.txt': {'Tamaño': 98, 'Densidad': 0.025036818851251842}, 'casa56.txt': {'Tamaño': 106, 'Densid

### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

     Noticia      Palabra  Centralidad de Grado  \
0  casa1.txt  feminicidio              0.165909   
1  casa1.txt       centro              0.052273   
2  casa1.txt    comercial              0.068182   
3  casa1.txt      santafe              0.029545   
4  casa1.txt        cinco              0.027273   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.159655                 0.425532   
1                       0.022777                 0.359184   
2                       0.041976                 0.372566   
3                       0.012639                 0.361248   
4                       0.011449                 0.349206   

   Centralidad de Vector Propio  
0                      0.394121  
1                      0.121389  
2                      0.149070  
3                      0.086139  
4                      0.069480  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/La Casa de la Mujer/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

           Centralidad de Grado                                          \
                          count      mean       std       min       25%   
Noticia                                                                   
casa1.txt                 441.0  0.019037  0.014818  0.006818  0.013636   
casa10.txt                251.0  0.029992  0.012701  0.012000  0.024000   
casa11.txt                149.0  0.051877  0.044107  0.027027  0.040541   
casa12.txt                 98.0  0.074690  0.046579  0.030928  0.061856   
casa13.txt                201.0  0.039005  0.021818  0.015000  0.030000   
...                         ...       ...       ...       ...       ...   
casa7.txt                 145.0  0.052778  0.024530  0.027778  0.041667   
casa70.txt                192.0  0.041012  0.022510  0.015707  0.031414   
casa71.txt                239.0  0.033086  0.016741  0.021008  0.025210   
casa8.txt                 256.0  0.028523  0.017519  0.011765  0.023529   
casa9.txt                

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para casa1.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.926407   
Centralidad de Cercania                    0.598317   
Centralidad de Vector Propio               0.857338   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.926407   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.518527   
Centralidad de Vector Propio                        0.737242   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.598317   
Centralidad de Intermediacion                 0.518527   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.815196   

                               Centralidad de Vector Propio  
Centralidad 

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

# PUBLIMETRO:

## Procesamiento, creación y exportación de gráficas

**MODELO DE PROCESAMIENTO POR NOTICIA**

In [ ]:
pip install unidecode

**IMPORTACIÓN Y PROCESAMIENTO DEL TEXTO, HISTOGRAMAS**

In [ ]:
# IMPORTACIÓN Y PREPROCESAMIENTO

import nltk
import string
import os
from collections import Counter
from unidecode import unidecode
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive    #SECCIÓN PARA MONTAR EL DIRECTORIO 'MEDIOS' DESDE EL DRIVE
drive.mount('/content/drive')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if not char.isdigit()])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [unidecode(word) for word in words]

    return words

def word_frequencies(words):
    return Counter(words)

folder_path = '/content/drive/MyDrive/Medios/Publimetro'  #Ruta del archivo que vamos a analizar (HAY QUE CREAR UN ACCESO DIRECTO DE LA CARPETA EN NUESTRA RESPECTIVA UNIDAD)

# Lista con todos los archivos .txt en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # Procesamiento de solo archivos .txt
        file_path = os.path.join(folder_path, file_name)  # Ruta completa del archivo
        text = read_text_file(file_path)
        words = preprocess_text(text)
        frequencies = word_frequencies(words)

        # Imprimir las 10 palabras más comunes del archivo actual
        #print(f'Archivo: {file_name}')
        #print(frequencies.most_common(10))

        # Graficar las 20 palabras más comunes del archivo actual
        labels, values = zip(*frequencies.most_common(20))
        plt.figure(figsize=(10, 5))  # Cambiar tamaño de la figura si tienes muchas palabras
        plt.bar(labels, values, color="pink")
        plt.xticks(rotation=90)
        plt.xlabel('Palabras')
        plt.ylabel('Frecuencia')
        plt.title(f'Frecuencia de palabras en {file_name}')
        #plt.show
        plt.close()

def histograma(frequencies):
  labels, values = zip(*frequencies.most_common(20))
  plt.bar(labels, values, color= "pink")
  plt.xticks(rotation=90)
  plt.xlabel('Palabras')
  plt.ylabel('Frecuencia')
  plt.title(f'Frecuencia de Palabras en {filename}')
  plt.savefig(f'/content/histograma_{filename}.png')
  plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**BIGRAMAS:**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from nltk.util import bigrams

def create_bigrams_graph(words,min_ocurrencias):
  G = nx.Graph()
  bigram_list = list(bigrams(words))
  for bigram in bigram_list:
      word1, word2 = bigram
      if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 1
      else:
          G.add_edge(word1, word2, weight=1)
  # Condición para el número de ocurrencia de las palabras
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  # Borrar nodos aislados/desconectados
  G.remove_edges_from(edges_to_remove)
  return G

# VARIABLE PARA EL NÚMERO DE OCURRENCIAS:
min_ocurrencias_b = 1

def plot_graph(words,filename):
  G = create_bigrams_graph(words,min_ocurrencias_b)
  # Dibujar el grafo
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, k=0.5)
  nx.draw_networkx_nodes(G, pos, node_color='pink', node_size=400, alpha=0.8)
  nx.draw_networkx_edges(G, pos, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.6)
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  # Título del grafo
  plt.title(f"Grafo de Bigramas de Palabras en {filename}")
  plt.axis('off')  # Quitar los ejes
  plt.savefig(f'/content/bigrama_{filename}.png')
  plt.close()  # Cerrar la figura para liberar memoria

# Diccionario para guardar los bigramas:
bigram_graphs = {}

**SKIP-GRAMAS**

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations


def create_skipgrams(words,skip,min_ocurrencias):
  G = nx.Graph()
  skipgram_list = []
  for i, word1 in enumerate(words):
    for j in range(i + 1, min(i + skip + 2, len(words))):
        word2 = words[j]
        skipgram_list.append((word1, word2))

  for skipgram in skipgram_list:
    word1, word2 = skipgram
    if G.has_edge(word1, word2):
        G[word1][word2]['weight'] += 1
    else:
        G.add_edge(word1, word2, weight=1)
  # Filtrar las aristas que tienen un peso menor que el mínimo establecido
  edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_ocurrencias]
  G.remove_edges_from(edges_to_remove)
  # Filtrar nodos desconectados o con grado 0
  nodes_to_remove = [node for node, degree in G.degree() if degree == 0]
  G.remove_nodes_from(nodes_to_remove)
  return G

# VARIABLES PARA DEFINIR EL NÚMERO DE OCURRENCIAS Y SKIP
min_ocurrencias_s = 1
skip = 2

def save_skipgram_graph(words,filename):
  G = create_skipgrams(words, skip, min_ocurrencias_s)
  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  pos = nx.fruchterman_reingold_layout(G, k=0.9, iterations=100)
  nx.draw_networkx_nodes(G, pos, node_size=400, node_color='pink')
  nx.draw_networkx_labels(G, pos, font_size=5, font_family='sans-serif')
  edges = G.edges(data=True)
  weights = [d['weight'] for (u, v, d) in edges]
  nx.draw_networkx_edges(G, pos, edgelist=edges, width=weights, alpha=0.8, edge_color='black')
  # Título del grafo
  plt.title(f'Grafo de Skipgramas de {filename}', fontsize=16)
  plt.savefig(f'/content/skip-grama_{filename}.png')
  plt.close()

# Diccionario para guardar los skipgramas:
skipgram_graphs = {}

**ITERACIÓN SOBRE TODOS LOS ARCHIVOS**: En esta sección llamamos a todas las funciones necesarias y se crean automáticamente las gráficas.

In [ ]:
import os
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
         filepath = os.path.join(folder_path, filename)
         print(filepath)
         text = read_text_file(filepath)
         words = preprocess_text(text)
         # Frecuencia de palabras e histograma
         frequencies = word_frequencies(words)
         histograma(frequencies)
         # Creación y almacenamiento de los grafos - bigramas
         Gb = create_bigrams_graph(words,min_ocurrencias_b)
         bigram_graphs[filename] = Gb
         # Creación y almacenamiento de los grafos - skipgramas
         Gs = create_skipgrams(words,skip,min_ocurrencias_s)
         skipgram_graphs[filename] = Gs
         # Creación de las imágenes
         plot_graph(words,filename)
         save_skipgram_graph(words,filename)

#print(read_text_file("/content/eltiempo9.txt"))

/content/drive/MyDrive/Medios/Publimetro/Publimetro53.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro34.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro42.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro41.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro5.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro46.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro44.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro45.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro47.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro48.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro50.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro43.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro51.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro52.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro9.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro55.txt
/content/drive/MyDrive/Medios/Publimetro/Publimetro54.txt
/content/drive/M

## Medidas de centralidad, densidad de la red, matrices varias:

### PARA LOS BIGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_bigramas = {}

for filename, G in bigram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_bigramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datab = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_bigramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datab.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_b = pd.DataFrame(datab)
# Verificar la estructura del DataFrame
#print(df_centralidades_b.head())

**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_b = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/Publimetro/Bigramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in bigram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_b[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_b = df_centralidades_b.groupby('Noticia').describe()
print(descriptivas_por_archivo_b)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
Eltiempo16.txt                130.0  0.020274  0.009956  0.007752  0.015504   
Eltiempo17.txt                234.0  0.011225  0.008498  0.004292  0.008584   
Eltiempo18.txt                151.0  0.016336  0.007763  0.006667  0.013333   
Eltiempo19.txt                224.0  0.011211  0.006755  0.004484  0.008969   
Eltiempo20.txt                170.0  0.016220  0.009619  0.005917  0.011834   
Eltiempo21.txt                215.0  0.012041  0.007118  0.004673  0.009346   
Eltiempo22.txt                221.0  0.012505  0.007602  0.004545  0.009091   
Eltiempo23.txt                134.0  0.019526  0.009317  0.007519  0.015038   
Eltiempo24.txt                 67.0  0.036183  0.013945  0.015152  0.030303   
eltiempo1.txt                 116.0  0.022039  0.011

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_b = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_b.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_b[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Eltiempo16.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.416827   
Centralidad de Cercania                    0.432189   
Centralidad de Vector Propio               0.718498   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.416827   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.248445   
Centralidad de Vector Propio                        0.184581   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.432189   
Centralidad de Intermediacion                 0.248445   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.612349   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_b = {}

for filename, G in bigram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_b[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }
print(tamano_densidad_b)

{'eltiempo1.txt': {'Tamaño': 116, 'Densidad': 0.022038980509745128}, 'eltiempo10.txt': {'Tamaño': 140, 'Densidad': 0.01778006166495375}, 'eltiempo11.txt': {'Tamaño': 276, 'Densidad': 0.00919631093544137}, 'eltiempo12.txt': {'Tamaño': 516, 'Densidad': 0.005599458116956424}, 'eltiempo13.txt': {'Tamaño': 551, 'Densidad': 0.005055271407358522}, 'eltiempo14.txt': {'Tamaño': 520, 'Densidad': 0.005180080035571365}, 'eltiempo15.txt': {'Tamaño': 154, 'Densidad': 0.01621254562431033}, 'eltiempo2.txt': {'Tamaño': 117, 'Densidad': 0.02180960801650457}, 'eltiempo3.txt': {'Tamaño': 276, 'Densidad': 0.010355731225296443}, 'eltiempo4.txt': {'Tamaño': 212, 'Densidad': 0.012697844943217384}, 'eltiempo5.txt': {'Tamaño': 224, 'Densidad': 0.011010570147341447}, 'eltiempo6.txt': {'Tamaño': 154, 'Densidad': 0.014854426619132501}, 'eltiempo7.txt': {'Tamaño': 150, 'Densidad': 0.017002237136465325}, 'eltiempo8.txt': {'Tamaño': 171, 'Densidad': 0.01582387340901273}, 'eltiempo9.txt': {'Tamaño': 243, 'Densidad': 0

### PARA LOS SKIPGRAMAS:

**MEDIDAS DE CENTRALIDAD**

In [ ]:
# Almacenar las centralidades en un diccionario
medidas_centralidad_skipgramas = {}

for filename, G in skipgram_graphs.items():
  # Centralidad de grado
  degree_centrality = nx.degree_centrality(G)
  # Centralidad de intermediación
  betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
  # Centralidad de cercanía
  closeness_centrality = nx.closeness_centrality(G)
  # Centralidad de vector propio
  eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
  # Impresión de medidas de centralidad (VERIFICACIÓN)
  #print(f'PARA {filename}')
  #print('Centralidad de grado:')
  #print(degree_centrality)
  #print('Centralidad de intermediación')
  #print(betweenness_centrality)
  #print('Centralidad de cercanía')
  #print(closeness_centrality)
  #print('Centralidad de vector propio')
  #print(eigenvector_centrality)

  #Guardar las centralidades en el diccionario
  medidas_centralidad_skipgramas[filename] = {
      "Grado": degree_centrality,
      "Intermediación": betweenness_centrality,
      "Cercanía": closeness_centrality,
      "VectorPropio": eigenvector_centrality
  }

# Creamos un DataFrame para hacer análisis estadístico:
datas = []
# Recorrer las medidas de centralidad para cada archivo y organizar los datos
for filename, centralidades in medidas_centralidad_skipgramas.items():
  # Convertir los diccionarios de centralidad en formato de DataFrame (pandas necesita listas o valores agregados)
  for node, centralidad_grado in centralidades['Grado'].items():
      centralidad_intermediacion = centralidades['Intermediación'].get(node, 0)
      centralidad_cercania = centralidades['Cercanía'].get(node, 0)
      centralidad_vector_propio = centralidades['VectorPropio'].get(node, 0)

      # Agregar la información del nodo y las centralidades al DataFrame
      datas.append({
          'Noticia': filename,
          'Palabra': node,
          'Centralidad de Grado': centralidad_grado,
          'Centralidad de Intermediacion': centralidad_intermediacion,
          'Centralidad de Cercania': centralidad_cercania,
          'Centralidad de Vector Propio': centralidad_vector_propio
      })
# Convertir la lista de diccionarios en un DataFrame
df_centralidades_s = pd.DataFrame(datas)
# Verificar la estructura del DataFrame
print(df_centralidades_s.head())

         Noticia  Palabra  Centralidad de Grado  \
0  eltiempo1.txt   bogota              0.226087   
1  eltiempo1.txt   envian              0.034783   
2  eltiempo1.txt   carcel              0.078261   
3  eltiempo1.txt   hombre              0.243478   
4  eltiempo1.txt  acusado              0.052174   

   Centralidad de Intermediacion  Centralidad de Cercania  \
0                       0.184585                 0.463710   
1                       0.000554                 0.368590   
2                       0.019923                 0.410714   
3                       0.250023                 0.477178   
4                       0.001721                 0.368590   

   Centralidad de Vector Propio  
0                      0.300635  
1                      0.078516  
2                      0.132639  
3                      0.265212  
4                      0.098199  


**MEDIDAS DE CENTRALIDAD - 25 PALABRAS MÁS RELEVANTES POR NOTICIA**

In [ ]:
# Creamos primero un diccionario
palabras_centrales_por_medida_s = {}

# Carpeta de salida con los .txt que enlistan los nodos más centrales
output_folder = '/content/drive/MyDrive/Medios/Resultados_nodos_centrales/Publimetro/Skipgramas'
os.makedirs(output_folder, exist_ok=True)

for filename, G in skipgram_graphs.items():
  # Creación de subcarpetas por noticias
  news_folder = os.path.join(output_folder, filename)
  os.makedirs(news_folder, exist_ok=True)
  # TOP 25 NODOS MÁS CENTRALES:
  # Centralidad de grado
  degree_centrality = centralidades["Grado"]
  top_25_grado = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de intermediación
  betweenness_centrality = centralidades["Intermediación"]
  top_25_intermediacion = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de cercanía
  closeness_centrality = centralidades["Cercanía"]
  top_25_cercania = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Centralidad de vector propio
  eigenvector_centrality = centralidades["VectorPropio"]
  top_25_vector_propio = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:25]
  # Diccionario - Top palabras más centrales
  palabras_centrales_por_medida_s[filename] = {
        "Grado": top_25_grado,
        "Intermediación": top_25_intermediacion,
        "Cercanía": top_25_cercania,
        "VectorPropio": top_25_vector_propio
  }
  # CREACIÓN DE LOS .TXT
  # Centralidad de grado
  with open(os.path.join(news_folder, f'centralidad_grado_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Grado para {filename}:\n\n")
      for nodo, valor in top_25_grado:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de intermediación
  with open(os.path.join(news_folder, f'centralidad_intermediacion_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Intermediación para {filename}:\n\n")
      for nodo, valor in top_25_intermediacion:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de cercanía
  with open(os.path.join(news_folder, f'centralidad_cercania_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Cercanía para {filename}:\n\n")
      for nodo, valor in top_25_cercania:
          f.write(f'{nodo}: {valor:.4f}\n')
  # Centralidad de vector propio
  with open(os.path.join(news_folder, f'centralidad_vector_propio_{filename}.txt'), 'w', encoding='utf-8') as f:
      f.write(f"Top 25 Nodos - Centralidad de Vector Propio para {filename}:\n\n")
      for nodo, valor in top_25_vector_propio:
          f.write(f'{nodo}: {valor:.4f}\n')

*ESTADÍSTICAS DESCRIPTIVAS Y MATRIZ DE CORRELACIONES ENTRE CENTRALIDADES*

In [ ]:
# Estadísticas descriptivas
descriptivas_por_archivo_s = df_centralidades_s.groupby('Noticia').describe()
print(descriptivas_por_archivo_s)

               Centralidad de Grado                                          \
                              count      mean       std       min       25%   
Noticia                                                                       
Eltiempo16.txt                130.0  0.060346  0.027690  0.023256  0.046512   
Eltiempo17.txt                234.0  0.033418  0.024051  0.012876  0.025751   
Eltiempo18.txt                151.0  0.049536  0.023845  0.020000  0.040000   
Eltiempo19.txt                224.0  0.034153  0.021527  0.013453  0.026906   
Eltiempo20.txt                170.0  0.048590  0.028346  0.017751  0.035503   
Eltiempo21.txt                215.0  0.036601  0.021983  0.014019  0.028037   
Eltiempo22.txt                221.0  0.037351  0.021910  0.013636  0.027273   
Eltiempo23.txt                134.0  0.059140  0.029788  0.022556  0.045113   
Eltiempo24.txt                 67.0  0.110357  0.043644  0.045455  0.090909   
eltiempo1.txt                 116.0  0.066417  0.034

In [ ]:
# Diccionario para almacenar las matrices de correlación por archivo
correlaciones_por_archivo_s = {}

# Agrupa el DataFrame por noticia y calcula la correlación para cada grupo
for filename, grupo in df_centralidades_s.groupby('Noticia'):
  # Seleccionar solo las columnas de centralidades
  centralidades = grupo[['Centralidad de Grado', 'Centralidad de Intermediacion', 'Centralidad de Cercania', 'Centralidad de Vector Propio']]
  # Calcular la matriz de correlación
  matriz_correlacion = centralidades.corr()
  # Almacenar la matriz de correlación en el diccionario
  correlaciones_por_archivo_s[filename] = matriz_correlacion
  # Mostrar la matriz de correlación para verificar (puedes eliminar esta línea si no la necesitas)
  print(f'Matriz de correlación para {filename}:')
  print(matriz_correlacion)

Matriz de correlación para Eltiempo16.txt:
                               Centralidad de Grado  \
Centralidad de Grado                       1.000000   
Centralidad de Intermediacion              0.577203   
Centralidad de Cercania                    0.542750   
Centralidad de Vector Propio               0.803592   

                               Centralidad de Intermediacion  \
Centralidad de Grado                                0.577203   
Centralidad de Intermediacion                       1.000000   
Centralidad de Cercania                             0.425142   
Centralidad de Vector Propio                        0.217614   

                               Centralidad de Cercania  \
Centralidad de Grado                          0.542750   
Centralidad de Intermediacion                 0.425142   
Centralidad de Cercania                       1.000000   
Centralidad de Vector Propio                  0.668132   

                               Centralidad de Vector Propio  
Central

**TAMAÑO Y DENSIDAD**

In [ ]:
tamano_densidad_s = {}

for filename, G in skipgram_graphs.items():
  sizzze = len(G)
  density = nx.density(G)
  tamano_densidad_s[filename] = {
      "Tamaño": sizzze,
      "Densidad": density
  }

In [ ]:
import shutil
import os

# Crear un directorio temporal para las imágenes
temp_dir = '/content/imagenes_temp'
os.makedirs(temp_dir, exist_ok=True)

# Mover todas las imágenes PNG al directorio temporal
for file_name in os.listdir('/content/'):
    if file_name.endswith('.png'):
        shutil.move(os.path.join('/content/', file_name), os.path.join(temp_dir, file_name))

# Comprimir todas las imágenes en el archivo .zip
shutil.make_archive('/content/imagenes_grafos', 'zip', temp_dir)

# Descargar el archivo .zip
from google.colab import files
files.download('/content/imagenes_grafos.zip')

# Eliminar el directorio temporal una vez comprimido
shutil.rmtree(temp_dir)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# FUNCION PARA COMBINAR TODAS LAS PALABRAS DE LOS ARCHIVOS .TXT EN UNA SOLA LISTA POR CARPETA
import os

def aggregate_words_from_folder(folder_path):
    all_words = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            text = read_text_file(file_path)
            words = preprocess_text(text)
            all_words.extend(words)  # Añadir todas las palabras a la lista
    return all_words

# Modificar para que se generen los grafos de bigramas y skipgramas de todas las palabras
for folder in ['/content/drive/MyDrive/Medios/El tiempo', '/content/drive/MyDrive/Medios/El espectador', '/content/drive/MyDrive/Medios/El colombiano']:
    all_words = aggregate_words_from_folder(folder)
    # Crear y guardar el grafo de bigramas para la carpeta
    G_bigram = create_bigrams_graph(all_words, min_ocurrencias_b)
    bigram_graphs[folder] = G_bigram
    plot_graph(all_words, folder.split('/')[-1] + "_bigramas")  # Guardar imagen

    # Crear y guardar el grafo de skipgramas para la carpeta
    G_skipgram = create_skipgrams(all_words, skip, min_ocurrencias_s)
    skipgram_graphs[folder] = G_skipgram
    save_skipgram_graph(all_words, folder.split('/')[-1] + "_skipgramas")  # Guardar imagen
